In [1]:
%cd ..
!ls

/exp/exp1/acp21rjf/deliberation/speachy
ami		    model_configs	   sweep.yaml
checkpoints	    model_utils.py	   tedlium
checkpoints_done    mwer_rescoring.py	   test_66_rescored.pkl
experiment_configs  non_iid_dataloader.py  tools.py
hyps		    pg191kwft.json	   top100_5kb_history_250.pkl
ipynbs		    __pycache__		   train_LM.py
IS_30s_utt.json     README.md		   train_lm.sh
IS_single_utt	    setup.py		   wandb
lm		    speachy		   wip
lm_utils.py	    speachy.egg-info
log.txt		    sweep_configs


In [2]:
from Levenshtein import distance

In [3]:
import pickle as pkl, numpy as np
from speachy.rescoring.tools import (
        sort_hypothesis_by_recording, 
        order_recordings_by_start_time,
        interpolate
)
from functools import partial, reduce
import torch

[NeMo W 2023-01-25 19:12:10 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-01-25 19:12:10 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [4]:
with open('hyps/test_beams_5000.pkl', 'rb') as f:
    test_beams = pkl.load(f)

In [5]:
test_beams = order_recordings_by_start_time(sort_hypothesis_by_recording(test_beams))

In [6]:
def get_edit_distance(hyps, target):
    return list(map(lambda x: distance(x, target) / len(target), hyps))
    
def create_samples_from_recording(recording, num_utterances, num_negatives, max_gap=10.0, shuffle=True):
    samples = []
    prev_end = None
    if shuffle == False:
        np.random.seed(42) # deterministic selection of negatives
    for i, utterance in enumerate(recording):
        start_t, end_t = utterance['meta_data']['timings'].values()
        if prev_end is None or (start_t - prev_end) > max_gap:
            samples.append([]) 
        if len(samples[-1]) >= num_utterances:
            samples.append([])
        hyps = utterance['beams'][0]
        hyps = list(map(lambda x: x['text'], list(hyps.values())))
        target = utterance['targets'][0]
        hyps = list(filter(lambda el:el != target, hyps))
        hyps = np.random.choice(hyps, min(num_negatives, len(hyps)), replace=False).tolist()
        examples = [target] + hyps
        error_rates = get_edit_distance(examples, target)
        samples[-1].append((examples, error_rates))
        prev_end = end_t
        
    return samples

In [7]:
def create_dataset_samples(recordings, num_utterances, num_negatives, max_gap=10.0, shuffle=True):
    samples = []
    for recording in recordings.keys():
        samples += create_samples_from_recording(recordings[recording], num_utterances, num_negatives, max_gap, shuffle)
    if shuffle:
        np.random.shuffle(samples) # shuffle samples
    return samples

In [96]:
def tokenize_and_pad(utterances, tokenizer): # returns padded utterances and their lengths
    tokenized = [tokenizer.text_to_ids(utt) for utt in utterances]
    max_len = max(map(len, tokenized))
    padded_utts = np.array([utt + [0] * (max_len - len(utt)) for utt in tokenized])
    return padded_utts, np.array(list(map(len, tokenized)))

In [97]:
flatten_nested_list = lambda l: [item for sublist in l for item in sublist]

In [146]:
def get_sub_batches(batch, tokenizer):
    proc_utts = lambda utts: tokenize_and_pad(flatten_nested_list(utts), tokenizer)
    sub_batches = []
    max_len = max(map(len, batch))
    for i in range(max_len):
        sub_batches.append({'utterances': [],'scores': [],'sb_lengths': [],})
        for el in batch:
            case = len(el) > i
            sub_batches[-1]['utterances'].append(el[i][0] if case else -1)
            sub_batches[-1]['scores'].append(el[i][1] if case else -1)
            sub_batches[-1]['sb_lengths'].append(len(el[i][0]) if case else -1)
     
    non_empty_indices = np.arange(len(sub_batches[0]['sb_lengths']))

    for i, sub_batch in enumerate(sub_batches):
        sb_utts, sb_scores, sb_lengths = [np.array(el, dtype=object) for el in (sub_batch['utterances'], sub_batch['scores'], sub_batch['sb_lengths'])]
        non_empty = non_empty_indices[sb_lengths != -1]
        # slice based on non empty of previous sub batch
        prev_fetch = None
        if i != 0:
            prev_lengths = sub_batches[i-1]['sb_lengths']
            prev_non_empty = sub_batches[i-1]['non_empty']
            diff_from = ((prev_lengths != -1) == (sb_lengths != -1))[prev_non_empty]
            prev_fetch = np.arange(len(prev_non_empty))[diff_from]
            
        sub_batches[i] = {
            'utterances': sb_utts,
            'scores': sb_scores,
            'sb_lengths': sb_lengths,
            'non_empty': non_empty,
            'prev_fetch': prev_fetch 
        }
    
    for i, sub_batch in enumerate(sub_batches):
        padded_utts, acc_lengths = proc_utts(sub_batch['utterances'][sub_batch['non_empty']].tolist())
        sub_batches[i] = {
            'utterances': torch.as_tensor(padded_utts),
            'lengths': torch.as_tensor(acc_lengths),
            'scores': torch.tensor(flatten_nested_list((sub_batch['scores'][sub_batch['non_empty']]).tolist())),
            'sb_lengths': torch.as_tensor(sub_batch['sb_lengths'][sub_batch['non_empty']].astype(int)),
            'prev_fetch': torch.as_tensor(sub_batch['prev_fetch']) if sub_batch['prev_fetch'].__class__.__name__ != 'NoneType' else None# indices to fetch the states from previous sub batch
        }
    return sub_batches

In [147]:
a = None
a.__class__.__name__

'NoneType'

In [148]:
class Sampler(object):
    def __init__(self, samples, batch_size, tokenizer, shuffle=True):
        self.samples = samples
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.shuffle = shuffle
        self.sample_indices = np.arange(len(self.samples))
        np.random.shuffle(self.sample_indices) if self.shuffle else None
        self.generator = self.__generator__()

    def __generator__(self):
        for i in range(0, len(self.samples), self.batch_size):
            yield get_sub_batches([self.samples[i] for i in self.sample_indices[i:i+self.batch_size]], self.tokenizer)

    def __iter__(self):
        return self

    def __len__(self):
        return len(self.samples)

    def __next__(self):
        return next(self.generator)



def sampler(samples, batch_size, tokenizer, shuffle=True):
    sample_indices = np.arange(len(samples))
    np.random.shuffle(sample_indices) if shuffle else None
    batches = []
    for i in range(0, len(samples), batch_size):
        yield get_sub_batches([samples[i] for i in sample_indices[i:i+batch_size]], tokenizer)
        

In [135]:
import tools

In [136]:
tk = tools.load_tokenizer('./tedlium/tokenizers/tokenizer_spe_bpe_v1000/tokenizer.model')

In [137]:
np.array([['f','z'],['helo']]).tolist()

[NeMo W 2023-01-25 19:27:42 nemo_logging:349] /tmp/ipykernel_1182549/3555533298.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
      np.array([['f','z'],['helo']]).tolist()
    


[['f', 'z'], ['helo']]

In [138]:
s1[-20]['prev_fetch']

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1182549/2132013132.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1182549/2132013132.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: list index out of range

In [139]:
s1[0].keys()

dict_keys(['utterances', 'lengths', 'scores', 'sb_lengths', 'prev_fetch'])

In [140]:
s1[-1]['scores']

tensor([0.0000, 0.0591, 0.0409, 0.0545, 0.0500, 0.0455, 0.0000, 0.0783, 0.0522,
        0.1304, 0.0783, 0.1043, 0.0000, 0.0000, 0.0926, 0.1296, 0.1296, 0.1111,
        0.0741, 0.0000, 0.0764, 0.1019, 0.1019, 0.1656, 0.0764, 0.0000, 0.0000,
        0.0889, 0.0815, 0.0667, 0.0741, 0.0889, 0.0000, 0.0635, 0.0741, 0.0370,
        0.0423, 0.0529, 0.0000, 0.0279, 0.0398, 0.0359, 0.0438, 0.0677])

In [141]:
torch.as_tensor(s1[-1]['lengths'].astype(int)).dtype

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1182549/1701062046.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1182549/1701062046.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Tensor' object has no attribute 'astype'

In [142]:
loader.beep = '0'

In [149]:
loader = Sampler(samples, 9, tk, shuffle=False)

In [153]:
s1[0]['sb_lengths']

tensor([6, 6, 6, 6, 6, 6, 6, 6, 6])

In [151]:
s1 = next(loader)

In [152]:
loader.batch_size

9

In [116]:
z[0]['utterances'].shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1182549/3858181404.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1182549/3858181404.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'z' is not defined

In [268]:
for i in loader:
    z=i
    break

In [244]:
next(loader)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_552617/606159215.py:1 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_552617/606159215.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'Sampler' object is not an iterator

In [17]:
loader = sampler(samples, 5, tk)
s1 = next(loader)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1182549/154326804.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1182549/154326804.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tk' is not defined

In [83]:
for el in loader:
    print(el[0]['utterances'].shape)

In [82]:
samples = create_dataset_samples(test_beams, num_negatives=5, num_utterances=5)

In [105]:
hyps

array(["i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sars handy never a' writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my theisarres handy never im writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for italian wired i always keep my the sars handy never are and writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my f the sars handy never ionm writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sarres handy i never a writing anything but"],
      dtype='<U188')

In [106]:
hyps = list(map(lambda el:el['text'],list(test_beams['AimeeMullins_2009P'][0]['beams'][0].values())))
target = test_beams['AimeeMullins_2009P'][0]['targets'][0]
hyps = list(filter(lambda el:el != target, hyps))
# select 5 random hyps
hyps = np.random.choice(hyps, min(5, len(hyps)), replace=False).tolist()
hyps = hyps + [target]

In [97]:
import torch

In [123]:
list(zip(get_edit_distance(hyps, target), hyps))

[(0.096045197740113,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my f sars handy i've never writing anything but"),
 (0.0903954802259887,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the f sars handy i never writing anything but"),
 (0.1016949152542373,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my f the sars handy i never i a and writing anything but"),
 (0.096045197740113,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my th the sars handy i never writing anything but"),
 (0.06779661016949153,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sars han

In [117]:
(torch.tensor(get_edit_distance(hyps, target))).mean(0)

tensor(0.0753)

In [115]:
(torch.tensor(get_edit_distance(hyps, target))) - (torch.tensor(get_edit_distance(hyps, target))).mean(0)

tensor([ 0.0207,  0.0151,  0.0264,  0.0207, -0.0075, -0.0753])